# Potrzebne pakiety i funkcje

In [1]:
from ucimlrepo import fetch_ucirepo

from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd

from UBPL import UBPL # metoda Uncertainty-Based Pseudo-Labeling
from make_pu_labels import make_pu_labels # funkcja do generowania danych PU

# Wczytanie danych

Dane *Breast Cancer Wisconsin (Diagnostic)* z repozytorium UCI:

In [2]:
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets
y = y.replace('M', 1)
y = y.replace('B', 0)
y = y.values.flatten()

# Podział na zbiór uczący i testowy

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/4)

# Metoda wyrocznia

Utworzenie regresji logistycznej z regularyzacją $\ell_2$ i parametrem regularyzacji $\lambda=1$, dopasowanie modelu do danych uczących i predykcja na zbiorze testowym:

In [4]:
logreg_oracle = LogisticRegression()
logreg_oracle.fit(X_train, y_train)
y_pred_oracle = logreg_oracle.predict(X_test)

Dodatkowo wyznaczenie estymatorów prawdopodobieństw a posteriori $y(x)$ na zbiorze uczącym:

In [5]:
y_pred_proba_oracle = logreg_oracle.predict_proba(X_train)[:,1]

Estymatory $\hat{y}(x)$ są wykorzystywane w funkcji *make_pu_labels* do generowania danych PU - są potrzebne przy wyznaczaniu funkcji skłonności w scenariuszu S2.

# Generowanie danych PU

- Funkcja *make_pu_labels* służy do generowania danych PU dla zadanej częstości etykiet $c$ i zadanego scenariusza - S1 lub S2.
- Dodatkowo konieczne jest podanie estymatorów $\hat{y}(x)$  na zbiorze uczącym.
- Funkcja zwraca zmienną zastępczą $S$ i wartości funkcji skłonności $e(x)$ dla każdej obserwacji ze zbioru uczącego PU.

Wygenerowanie danych PU z wykorzystaniem schematu S1 i częstości etykiet $c=0.4$:

In [6]:
s_S1, ex_true_S1 = make_pu_labels(X_train, y_train, prob_true=y_pred_proba_oracle, label_scheme='S1', c=0.4)

Analogicznie można wygenerować dane PU z wykorzystaniem schematu S2 i częstości etykiet $c=0.7$:

In [7]:
s_S2, ex_true_S2 = make_pu_labels(X_train, y_train, prob_true=y_pred_proba_oracle, label_scheme='S2', c=0.7)

# Metoda naiwna

Przykład dla danych PU wygeberowanych z wykorzystaniem schematu S1 i częstości etykiet $c=0.4$.

Utworzenie regresji logistycznej z regularyzacją $\ell_2$ i parametrem regularyzacji $\lambda=1$, dopasowanie modelu do danych uczących PU i predykcja na zbiorze testowym:

In [8]:
logreg_naive = LogisticRegression()
logreg_naive.fit(X_train, s_S1)
y_pred_naive = logreg_naive.predict(X_test)

# Metoda Uncertainty-Based Pseudo-Labeling (UBPL)

Przykład dla danych PU wygeberowanych z wykorzystaniem schematu S1 i częstości etykiet $c=0.4$.

Utworzenie modelu UBPL dla 5 modeli logistycznych w komitecie klasyfikatorów (parametr $K$), 20 iteracji (parametr $J$), progu dolnego $t_l=0.05$ i progu górnego $t_u=0.35$:

In [9]:
ubpl = UBPL(K=5, J=20, t_l=0.05, t_u=0.35)

Dopasowanie modelu do danych PU:

In [10]:
ubpl.fit(X_train, s_S1)

Wyznaczenie predykcji, tzn. estymatora zmiennej odpowiedzi na zbiorze uczącym:

In [11]:
y_ubpl = ubpl.predict()

Utworzenie regresji logistycznej z regularyzacją $\ell_2$ i parametrem regularyzacji $\lambda=1$, dopasowanie modelu do danych uczących z zastosowaniem estymatora zmiennej odpowiedzi wyznaczonego w poprzednim kroku oraz predykcja na zbiorze testowym:

In [12]:
logreg_ubpl = LogisticRegression()
logreg_ubpl.fit(X_train, y_ubpl)
y_pred_ubpl = logreg_ubpl.predict(X_test)

# Porównanie wyników 3 metod z wykorzystaniem zbalansowanej dokładności na zbiorze testowym

Zbalansowana dokładność dla metody wyroczni:

In [13]:
print(balanced_accuracy_score(y_test, y_pred_oracle))

0.9168684422921711


Zbalansowana dokładność dla metody naiwnej:

In [14]:
print(balanced_accuracy_score(y_test, y_pred_naive))

0.5872679580306699


Zbalansowana dokładność dla metody UBPL:

In [15]:
print(balanced_accuracy_score(y_test, y_pred_ubpl))

0.9058716707021792


Jak widać w tym przypadku metoda UBPL znacząco poprawia wyniki w stosunku do metody naiwnej, a zbalansowana dokładność dla UBPL jest zbliżona do zbalansowanej dokładności dla metody wyroczni.